This notebook goes over all of the major components of this project:
1. Simulation: Can be thought of as a black box that takes in a solution and outputs demand coverage. Used to evaluate solutions and generate a dataset.
2. Machine learning: A multilayer perceptron (MLP) is trained to predict coverage given a solution.
3. Optimization: The MLP is embedded within a MIP which attempts to find the solution that the MLP predicts will have the highest coverage.

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from ems_data import EMSData, TORONTO_AVG_CALLS_PER_DAY, TORONTO_N_AMBULANCES
from simulation import Simulation
from neural_network import MLP
from mip_models import *

Before doing anything, we need to load the data. The EMSData class reads and preprocesses the data for usage in simulation and MIP models.

In [2]:
# Skip this cell if you already have ems_data.pkl and don't need to regenerate with different parameters
ems_data = EMSData(region_id=1, x_intervals=10, y_intervals=10, verbose=True)
ems_data.save_instance('ems_data.pkl')

Computing travel times: 100%|██████████| 13994200/13994200 [01:06<00:00, 211712.25it/s]


In [3]:
ems_data = EMSData.load_instance('ems_data.pkl')

n_stations = len(ems_data.stations)
n_demand_nodes = len(ems_data.demand_nodes)
demand = ems_data.demand_nodes.demand
print(f"# stations: {n_stations}")
print(f"# demand nodes: {n_demand_nodes}")

# stations: 46
# demand nodes: 67


# Simulation
The Simulation class pulls relevant data from an EMSData instance and runs the simulation. Simulations are used to evaluate a solution (i.e., the number of ambulances at each station) as well as generate a dataset for the MLP.

In [4]:
# Pickled Simulation instance is used by HTCondor jobs
#sim = Simulation(data=ems_data, avg_calls_per_day=TORONTO_AVG_CALLS_PER_DAY, n_days=100, n_replications=10)
sim = Simulation(data=ems_data, avg_calls_per_day=1000, n_days=100, n_replications=10)
sim.save_instance('simulation.pkl')

In [5]:
# Run multiple replications and evaluate coverage
def evaluate_solution(solution):
    sim_result = sim.run(solution)
    result = sim_result.sum()  # Sum over replications
    result = np.array([result[f'covered{i}']/result[f'total{i}'] for i in range(n_demand_nodes)])
    result = np.nan_to_num(result, nan=0.0)
    coverage = demand@result / demand.sum()  # Estimated long-term coverage
    return coverage

# Evaluate the solution that places 5 ambulances at each station
evaluate_solution([5]*n_stations)

0.9508725812351208

In [6]:
# Evaluate the solution that has unlimited ambulances at each station (1000 is basically unlimited)
evaluate_solution([1000]*n_stations)

0.9513710152281083

The dataset was generated using HTCondor and is stored in `dataset.csv`. `n_jobs = 100` jobs are run, each job performs the simulation for `solutions_per_job = 1000` solutions, and `n_replications = 10` replications are ran per solution. The resulting dataset has `n_jobs * solutions_per_job` samples, one per solution (the `n_replications` replications for a solution are aggregated into a single sample).

To generate `dataset.csv`:
1. Run `htcondor_setup.py` to generate `settings<Process>.csv` files. These files contain the solutions to be simulated on each HTCondor job.
2. Move the following files to the HTCondor submit server:
    - `simulation.sub`
    - `run_job.sh`
    - `run_job.py`
    - `simulation.py`
    - `settings$(Process).csv` for each `Process`
    - `simulation.pkl`
    - `sim-env.tar.gz` (see https://chtc.cs.wisc.edu/uw-research-computing/conda-installation, Option 1; the environment must have numpy and pandas)

    The last two files go to your Squid directory (see https://chtc.cs.wisc.edu/uw-research-computing/file-avail-squid).
    
3. Run `condor_submit simulation.sub`. Once the jobs are done, move the `results<Process>.csv` files to a new folder named `sim_results`. Then run `create_dataset.py` to create the dataset from the `results<Process>.csv` files. For each solution, the script sums the `covered<i>` and `total<i>` columns over the replications and defines `coverage<i>` as the ratio of the two sums. The resulting dataset has columns `station<i>` for `i` in `range(n_stations)`, and `coverage<i>` for `i` in `range(n_demand_nodes)`.

In [7]:
dataset = pd.read_csv('dataset.csv')
dataset

,station0,station1,station2,station3,station4,station5,station6,station7,station8,station9,...,coverage57,coverage58,coverage59,coverage60,coverage61,coverage62,coverage63,coverage64,coverage65,coverage66
0,12,7,1,1,2,0,3,0,4,0,...,0.927025,0.905187,0.919772,0.868613,0.969797,0.914167,0.955526,0.781250,0.666667,0.857143
1,7,14,2,6,2,6,11,6,10,3,...,0.893645,0.902134,0.923345,0.925000,0.834316,0.858432,0.923382,0.718750,1.000000,0.704545
2,7,4,3,19,5,3,6,11,4,1,...,0.915328,0.906570,0.924791,0.895652,0.964265,0.903403,0.920114,0.727273,0.666667,0.647059
3,4,2,25,5,1,5,1,4,3,3,...,0.930064,0.912834,0.924211,0.919118,0.958333,0.906962,0.955671,0.693878,1.000000,0.700000
4,4,5,0,5,3,18,9,3,1,11,...,0.931480,0.908427,0.924083,0.886525,0.970378,0.901681,0.904494,0.617647,1.000000,0.773585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2,7,9,0,1,1,12,14,0,2,...,0.926924,0.893367,0.928029,0.924370,0.956327,0.908037,0.947090,0.862069,0.804716,0.853659
99996,3,16,5,8,5,9,11,1,3,18,...,0.925392,0.914516,0.932284,0.872881,0.956100,0.914417,0.940415,0.750000,1.000000,0.725000
99997,0,1,1,0,5,10,2,8,2,0,...,0.924575,0.901637,0.927235,0.867257,0.964025,0.908904,0.944518,0.756757,0.600000,0.777778
99998,1,3,4,12,9,4,8,3,2,6,...,0.922358,0.907812,0.928703,0.942623,0.971351,0.901662,0.907552,0.666667,1.000000,0.861111


# Machine Learning
The MLP takes as input the solution and outputs the coverage probabilities for each demand node.

In [8]:
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

NVIDIA GeForce GTX 1070


device(type='cuda')

In [9]:
# Move dataset into tensors compatible with model and split into train/dev/test sets
X = dataset[[f'station{i}' for i in range(n_stations)]].to_numpy()
y = dataset[[f'coverage{i}' for i in range(n_demand_nodes)]].to_numpy()
X = torch.tensor(X, dtype=torch.float32, device=device)
y = torch.tensor(y, dtype=torch.float32, device=device)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.75)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, random_state=0, train_size=2/3)
X_train.shape, X_dev.shape, X_test.shape, y_train.shape, y_dev.shape, y_test.shape

(torch.Size([50000, 46]),
 torch.Size([25000, 46]),
 torch.Size([25000, 46]),
 torch.Size([50000, 67]),
 torch.Size([25000, 67]),
 torch.Size([25000, 67]))

In [10]:
# Train MLP
weights = torch.tensor(demand, dtype=torch.float32, device=device)
loss_fn = lambda logits, targets: MLP.regression_loss(logits, targets, weights, sum_outputs=True, modified_sigmoid=True)

mlp = MLP(n_stations, [200], n_demand_nodes, dropout=0.1).to(device)
init_train_loss = mlp.evaluate_loss(X_train, y_train, loss_fn)
init_dev_loss = mlp.evaluate_loss(X_dev, y_dev, loss_fn)
print(f"Initial train loss: {init_train_loss}, initial dev loss: {init_dev_loss}")
mlp.fit(X_train, y_train, X_dev, y_dev, loss_fn, tolerance=0.1, verbose=True)

Evaluating:   0%|          | 0/391 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 196/196 [00:00<00:00, 809.92it/s]


Initial train loss: 18439157788.18048, initial dev loss: 18420646394.59328


Evaluating: 100%|██████████| 196/196 [00:00<00:00, 750.97it/s]


Train loss: 142479357.7836, dev loss: 1559577.81246


...

Train loss: 148855.5422825, dev loss: 370615.00579


Evaluating: 100%|██████████| 196/196 [00:00<00:00, 618.21it/s]


Train loss: 146046.67472, dev loss: 458124.10082


Evaluating: 100%|██████████| 196/196 [00:00<00:00, 638.44it/s]

Train loss: 141400.636175, dev loss: 515183.58158
Early stopping


In [11]:
# Load a saved model
mlp = MLP.load_model('model.pt').to(device)
mlp

MLP(
  (0): Linear(in_features=46, out_features=200, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.0, inplace=False)
  (3): Linear(in_features=200, out_features=67, bias=True)
)

# Optimization
The trained MLP is embedded within a MIP which attempts to find the solution that the MLP predicts will have the highest coverage.

In [12]:
# Daskin's MEXCLP model
coverage = compute_coverage(ems_data)
solution = mexclp(demand=demand, coverage=coverage, n_ambulances=TORONTO_N_AMBULANCES, busy_fraction=0.5, time_limit=60, verbose=True)
print(f"MEXCLP solution: {solution}\nTotal ambulances used: {sum(solution)}")
score = evaluate_solution(solution)
print(f"Long-term coverage of MEXCLP: {score}")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-17


Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 5 3600 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 68 rows, 15724 columns and 16702 nonzeros
Model fingerprint: 0x32c83fdc
Variable types: 0 continuous, 15724 integer (15678 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-71, 1e+04]
  Bounds range     [1e+00, 2e+02]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 0 rows and 12807 columns
Presolve time: 0.02s
Presolved: 68 rows, 2917 columns, 3687 nonzeros
Variable types: 0 continuous, 2917 integer (2883 binary)
Found heuristic solution: objective 108718.00000

Root relaxation: objective 2.977000e+05, 176 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Une

In [13]:
# Our model with MLP embedded
solution = mexclp_mlp(demand=demand, mlp=mlp, n_ambulances=TORONTO_N_AMBULANCES, time_limit=60, verbose=True)
print(f"Our model's solution: {solution}\nTotal ambulances used: {sum(solution)}")
score = evaluate_solution(solution)
print(f"Long-term coverage of our model: {score}")

Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 5 3600 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1651 rows, 826 columns and 25879 nonzeros
Model fingerprint: 0xd715233b
Variable types: 580 continuous, 246 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e-06, 4e+02]
  Objective range  [1e+00, 2e+04]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e-05, 4e+02]
Found heuristic solution: objective 157354.85726
Presolve removed 46 rows and 46 columns
Presolve time: 0.04s
Presolved: 1605 rows, 780 columns, 25787 nonzeros
Variable types: 534 continuous, 246 integer (200 binary)

Root relaxation: objective 2.977000e+05, 1532 iterations, 0.08 seconds (0.15 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd  

In [14]:
# Our model with MLP embedded, n_ambulances=1000 to see if model makes use of all ambulances
solution = mexclp_mlp(demand=demand, mlp=mlp, n_ambulances=1000, time_limit=60, verbose=True)
print(f"Our model's solution: {solution}\nTotal ambulances used: {sum(solution)}")
score = evaluate_solution(solution)
print(f"Long-term coverage of our model: {score}")

Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 5 3600 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1651 rows, 826 columns and 25879 nonzeros
Model fingerprint: 0x1c5a39ab
Variable types: 580 continuous, 246 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e-06, 2e+03]
  Objective range  [1e+00, 2e+04]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e-05, 2e+03]
Found heuristic solution: objective 157354.85726
Presolve removed 46 rows and 46 columns
Presolve time: 0.04s
Presolved: 1605 rows, 780 columns, 25787 nonzeros
Variable types: 534 continuous, 246 integer (200 binary)

Root relaxation: objective 2.977000e+05, 678 iterations, 0.02 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   

We observe that when the neural net is trained on samples each using exactly 234 total ambulances, increasing `n_ambulances` beyond 234 does not help us.
- The function mapping solutions to overall coverage should be monotone: if $x \le \hat{x}$, then $\hat{x}$ should perform at least as well as $x$. Does the neural net behave like this?
- Some stations are simply better than others. Since all samples $(x, y)$ in the dataset have $x$ summing to 234, placing more ambulances at "bad" stations means taking ambulances away from "good" stations. Does this mean the neural net is actually learning which stations are bad, and if so, would increasing the input for a bad station reduce the output? 

The goal of this research is to compete with other optimization models for the ambulance location problem, such as Daskin's MEXCLP. We want to show the disconnect between other models (specifically their objective function) and reality (i.e., simulation).
- What we are NOT trying to do: use ML to solve MEXCLP.
- What we ARE trying to do: use ML to model reality (i.e., as a surrogate for the simulation).

Other notes:
- The Muskoka instance (8 (actually 5, original code produced 8) stations, 62 demand nodes, 30 ambulances) may not be that interesting: only around 10M solutions, and the neural net doesn't reveal any solutions that significantly outperform the best simulation solution. The best simulation solution and our model's solution are comparable and outperform Daskin's MEXCLP; since stations are evenly spaced, the assumptions Daskin makes are reasonable, so outperforming MEXCLP is a big win.
- We can attack the problem from either the ML side (improving the neural net, e.g., architecture, loss function, training) or the MIP side. Is there something akin to regularization but for the MIP model?
    - Use MEXCLP's objective as a feature to the ML model?
- Applications to queueing?
- Decision-aware learning/smart predict then optimize?